In [ ]:
import json
import os
import time
from typing import List, Dict, Any

from langchain_core.documents import Document
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.prompts import PromptTemplate
from langchain_huggingface import HuggingFaceEmbeddings
from pinecone import Pinecone, ServerlessSpec
from pinecone_text.sparse import BM25Encoder
import torch
from utils import preprocess_text 
from dotenv import load_dotenv

load_dotenv()


: 

In [ ]:
def load_json_files(folder_path: str) -> List[Document]:
    all_docs = []
    chunk_counter = 1

    for filename in os.listdir(folder_path):
        if not filename.endswith(".json"):
            continue

        file_path = os.path.join(folder_path, filename)
        try:
            with open(file_path, "r", encoding="utf-8") as f:
                data = json.load(f)

            if "chunks" not in data:
                print(f" {filename}: Missing 'chunks' key — skipped")
                continue

            for chunk in data["chunks"]:
                content = chunk.get("content", "").strip()
                metadata = {
                    "file": filename,
                    "chunk_id": str(chunk_counter),
                    "document_type": chunk.get("document_type", ""),
                    "section": chunk.get("section", ""),
                    "source_type": chunk.get("source_type", ""),
                    "source_link": chunk.get("source_link", ""),
                }

                # Flatten subsections, tags, refs
                for key in ["subsection", "tag", "references"]:
                    value = chunk.get(key, [])
                    if isinstance(value, list):
                        for i, v in enumerate(value):
                            metadata[f"{key}_{i+1}"] = str(v)
                    else:
                        metadata[f"{key}_1"] = str(value)

                all_docs.append(Document(page_content=content, metadata=metadata))
                chunk_counter += 1

        except Exception as e:
            print(f" Error in {filename}: {e}")

    print(f" Loaded {len(all_docs)} documents from {folder_path}")
    return all_docs

In [ ]:
# Pinecone Index Creation (Hybrid)
def create_pinecone_hybrid_index(index_name: str, dimension: int, api_key: str,
                                  environment: str = "us-east-1"):
    pc = Pinecone(api_key=api_key)
    existing_indexes = [i.name for i in pc.list_indexes()]

    if index_name in existing_indexes:
        print(f" Index '{index_name}' already exists.")
        return pc.Index(index_name)

    print(f"Creating Pinecone HYBRID index '{index_name}'...")
    pc.create_index(
        name=index_name,
        dimension=dimension,
        metric="dotproduct", 
        spec=ServerlessSpec(cloud="aws", region=environment),
    )

    while not pc.describe_index(index_name).status["ready"]:
        print(" Waiting for index to be ready...")
        time.sleep(1)

    print(f" Hybrid index '{index_name}' created successfully.")
    return pc.Index(index_name)

# Batched Hybrid Uploader
def upload_hybrid_in_batches(documents: List[Document],
                            dense_embeddings,
                            bm25_encoder: BM25Encoder,
                            index_name: str,
                            api_key: str,
                            environment: str = "us-east-1",
                            batch_size: int = 50):
    """
    Upload documents with both dense and sparse vectors to Pinecone.
    
    Args:
        documents: List of LangChain Document objects
        dense_embeddings: HuggingFace embeddings model
        bm25_encoder: Fitted BM25Encoder for sparse vectors
        index_name: Pinecone index name
        api_key: Pinecone API key
        environment: Pinecone environment
        batch_size: Number of documents per batch
    """
    pc = Pinecone(api_key=api_key)
    
    # Create index
    dimension = len(dense_embeddings.embed_query("test"))
    index = create_pinecone_hybrid_index(index_name, dimension, api_key, environment)

    total = len(documents)
    total_batches = (total + batch_size - 1) // batch_size
    print(f" Uploading {total} docs with HYBRID vectors in {total_batches} batches...")

    for i in range(0, total, batch_size):
        batch = documents[i:i + batch_size]
        batch_no = i // batch_size + 1
        print(f" Batch {batch_no}/{total_batches} — {len(batch)} docs")

        try:
            # Prepare vectors for this batch
            vectors_to_upsert = []
            
            for doc in batch:
                # Generate dense vector
                dense_vector = dense_embeddings.embed_query(doc.page_content)
                
                # Generate sparse vector
                sparse_vector = bm25_encoder.encode_documents([doc.page_content])[0]
                
                # Prepare vector entry
                vector_entry = {
                    "id": doc.metadata["chunk_id"],
                    "values": dense_vector,
                    "sparse_values": sparse_vector,
                    "metadata": {
                        **doc.metadata,
                        "text": doc.page_content  # Store text for retrieval
                    }
                }
                vectors_to_upsert.append(vector_entry)
            
            # Upsert to Pinecone
            index.upsert(vectors=vectors_to_upsert)
            print(f" Uploaded batch {batch_no} with hybrid vectors")
            
        except Exception as e:
            print(f" Error uploading batch {batch_no}: {e}")
            print(" Retrying in 5s...")
            time.sleep(5)
            continue

        # Release memory
        if torch.cuda.is_available():
            torch.cuda.empty_cache()
        time.sleep(2)

    print(" All batches with hybrid vectors uploaded successfully!")


In [ ]:
PINECONE_API_KEY = os.getenv("PINECONE_API_KEY")
PINECONE_ENVIRONMENT = "us-east-1"
INDEX_NAME = "nepali-docs-hybrid"
DATA_PATH = "../../data/processed/"

if not PINECONE_API_KEY:
    raise ValueError("Please set PINECONE_API_KEY environment variable")

print("Loading documents...")
docs = load_json_files(DATA_PATH)

print("Loading dense embedding model...")
dense_embeddings = HuggingFaceEmbeddings(
    model_name="universalml/Nepali_Embedding_Model"
    # model_name="sentence-transformers/paraphrase-multilingual-mpnet-base-v2"
)

print("Fitting BM25 encoder for sparse vectors...")
bm25_encoder = BM25Encoder()
corpus = []

for doc in docs:
    processed = preprocess_text(doc.page_content)
    corpus.append(processed)

bm25_encoder.fit(corpus)
# Save the fitted BM25 encoder for later use
bm25_encoder.dump("bm25_params.json")
print("BM25 encoder saved to bm25_params.json")

print("\n Starting hybrid upload...")
upload_hybrid_in_batches(
    documents=docs,
    dense_embeddings=dense_embeddings,
    bm25_encoder=bm25_encoder,
    index_name=INDEX_NAME,
    api_key=PINECONE_API_KEY,
    environment=PINECONE_ENVIRONMENT,
    batch_size=50
)

print("🏁 Pinecone setup complete!")

Loading documents...
✅ Loaded 102 documents from ../../data/processed/
Loading dense embedding model...


c:\Users\thebi\Desktop\DocsGuide\src\projenv\Lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\thebi\.cache\huggingface\hub\models--sentence-transformers--paraphrase-multilingual-mpnet-base-v2. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
Xet Storage is enabled for this repo, but the 'hf_xet' package i

Fitting BM25 encoder for sparse vectors...


100%|██████████| 102/102 [00:00<00:00, 596.56it/s]


BM25 encoder saved to bm25_params.json

🚀 Starting hybrid upload...
Creating Pinecone HYBRID index 'nepali-docs-hybrid-multilingual'...
✅ Hybrid index 'nepali-docs-hybrid-multilingual' created successfully.
📦 Uploading 102 docs with HYBRID vectors in 3 batches...
➡️ Batch 1/3 — 50 docs
✅ Uploaded batch 1 with hybrid vectors
➡️ Batch 2/3 — 50 docs
✅ Uploaded batch 2 with hybrid vectors
➡️ Batch 3/3 — 2 docs
✅ Uploaded batch 3 with hybrid vectors
🎉 All batches with hybrid vectors uploaded successfully!
🏁 Pinecone setup complete!


In [ ]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained('sentence-transformers/LaBSE')
tokens = tokenizer("नेपाली नागरिकता प्राप्त गर्नका लागि नयाँ प्रक्रिया अन्तर्गत आवश्यक पर्ने विभिन्न कागजात तथा सिफारिसहरूको सूची निम्न अनुसार रहेको छ। यी कागजातहरू विभिन्न अवस्थाहरूमा अनिवार्य हुन सक्छन्। अनुसूची-१ को सिफारिस फाराममा वडा अध्यक्षको सिफारिस। अनुसूची-१ मा टाँस भए जस्तै २ प्रति फोटो। विवाह दर्ता प्रमाण पत्र। पति, बुवा, आमा, दाजु, भाई र छोराको नागरिकता। जन्मदर्ता प्रमाण-पत्र वा जन्म स्थान र जन्म मिति एकिन हुने गरी वडा कार्यालयबाट प्रमाणित गरिएको सिफारिस पत्र। वडा मुचुल्का सिफारिस पत्र (नाता पर्ने बाहेकका मुचुल्कामा बस्ने व्यक्तिहरूको नाम खुलेको)। प्रहरी मुचुल्का। बुवा, आमा, पति र पत्नीको मृत्यु भएको हकमा मृत्यु दर्ता प्रमाण पत्र। बसाई सरी आएको भए सम्बन्धित वडाको स्थानीय पञ्जिकाधिकारीबाट जारी बसाईसराई प्रमाणपत्र। सनाखत गर्ने एकाघरको व्यक्तिसँगको नाता प्रमाणित पत्र। बुवा/आमाबाट सनाखत (बुवा/आमा बाहेक अन्य एकाघरका सदस्यबाट सनाखत गर्नुपर्ने हकमा नाता प्रमाणित प्रमाण-पत्र)। विवाहित महिलाको हकमा पतिको सनाखत, पति नभई सासु/ससुरा/जेठाजु/देवरबाट सनाखत गर्नुपर्ने हकमा नाता प्रमाणित प्रमाण-पत्र। ना. प्र. प. को दुवै प्रति र नागरिकताको फोटोकपी, मतदाता परिचय पत्र, सवारी चालक अनुमति पत्र, जग्गाधनी प्रमाण पुर्जा र संघ संस्था वा सरकारी निकायको परिचय पत्र। सम्बन्ध विच्छेद भएको अवस्थामा सम्बन्ध विच्छेद प्रमाण-पत्र, अदालतको फैसलाको प्रतिलिपि र वडा कार्यालयको सिफारिस पत्र। नाम, थर वा अन्य विवरण फरक परेको हकमा फरक परेको व्यक्ति एउटै भएको भन्ने व्यहोराको वडा कार्यालयको सिफारिस पत्र।", return_tensors='pt', truncation=False)
print(tokens['input_ids'].shape)

c:\Users\thebi\Desktop\DocsGuide\src\projenv\Lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\thebi\.cache\huggingface\hub\models--sentence-transformers--LaBSE. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


torch.Size([1, 346])
